In [26]:
import pandas as pd
from to_db import Connector

### The following cell pulls the temperature data from the database and group by date 

In [103]:
conn = Connector()
df = conn.get_data()
df["date"] = pd.to_datetime(df['date'])
data = df.groupby("date").mean()

In [124]:
from datetime import datetime

In [99]:
from time import localtime,strftime,time

In [86]:
td = datetime.now()

In [106]:
today = df.loc[df['date'] == td.date()]
daily = today

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.



In [73]:
import serial

In [74]:
from numpy import median

In [75]:
def date_range():
    obj = dcc.DatePickerSingle(
        id='date_picker',
        min_date_allowed = data.index[0],
        max_date_allowed = date.today(),
        initial_visible_month = date.today(),
        date = date.today()
    )
    
    return obj

In [76]:
def statistics():
    obj = dcc.RadioItems(
        id = "stats",
        options=[
            {'label': 'max','value':'max'},
            {'label': 'min','value':'min'},
            {'label': 'mean','value':'mean'}],
        value = 'mean'
    )
    
    return obj

In [77]:
ser = serial.Serial('/dev/ttyACM0')
ser.flushInput()

In [78]:
READINGS = 20

In [79]:
def read_temp(ser):
    while True:
        readings = []
        try:
            for i in range(READINGS):
                bytes = ser.readline()
                decoded = bytes[0:len(bytes)-2].decode("utf-8")
                decoded = float(decoded)
                readings.append(decoded)
            break
        except:
            pass

    return median(decoded)

In [80]:
temp  = read_temp(ser)
old = None

In [122]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import date



stats = {"mean": df.groupby("date").mean(),"max":df.groupby("date").max(),"min":df.groupby("date").min()}


app = JupyterDash(__name__)
colours = {'text': '#7FDBFF', 'background':'#777777'}
text_size = {'H1':48,'H2':40,'text':36}

# fig = make_subplots(rows = 1, cols= 2)

# fig.add_trace(go.Scatter(x = data.index, y = data["temperature"]),row=1,col=1)
# fig.add_trace(go.Scatter(x= today["time"], y= today["temperature"]),row=1,col=2)

fig = px.scatter(x = data.index, y = data["temperature"])
# fig = px.scatter(df, x = "date" , y = "temperature")

fig.update_yaxes(rangemode = "tozero")

fig2 = go.Figure(go.Indicator(mode="gauge+number",value=temp,title={'text':'Temperature/C'}))

fig3 = px.scatter(daily,"time", "temperature")
fig3.update_yaxes(rangemode = "tozero")

app.layout = html.Div(style={'backgroundColor':colours['background'],'fontFamily':'calibri'}, children=[

    html.H1(children='Room Temperature',
        style = {'textAlign': 'center',
                 'color':colours['text'],
                 'fontSize':text_size['H1']}),

    dcc.Graph(
        id='graph2',
        figure=fig2
    ),
    
    
    statistics(),
    

    dcc.Graph(
        id='graph',
        figure=fig
    ),
    
    date_range(),

    
    dcc.Graph(
        id='graph3',
        figure=fig3
    )
])

@app.callback(
    Output('graph2','figure'),
    Output('graph','figure'),
    Output('graph3','figure'),
    Input('stats','value'),
    Input('date_picker','date'))
def update_figure(stat,day):
    temp  = read_temp(ser)
    data = stats[stat]
    day = datetime.strptime(day,"%Y-%m-%d")
#     print(day)
    
    daily = df.loc[df['date'] == day.date()]
    
    old = temp
#     fig = px.scatter(df, x = "date" , y = "temperature")

    fig = px.scatter(x = data.index, y = data["temperature"])
    fig.update_yaxes(rangemode = "tozero")
    fig2 = go.Figure(go.Indicator(mode="gauge+number",value=temp,title={'text':'Temperature/C'},delta={'reference':old}))

    fig3 = px.scatter(daily,"time", "temperature")
    fig3.update_yaxes(rangemode = "tozero")
#     fig.data =[]

#     fig.add_trace(go.Scatter(x = data.index, y = data["temperature"]),row=1,col=1)
#     fig.add_trace(go.Scatter(x=today["time"], y=today["temperature"]),row=1,col=2)
#     fig.update_layout(showlegend=False)
    
    return fig2,fig,fig3

In [123]:
app.run_server(mode = "external")

/home/blunt/.local/lib/python3.7/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://127.0.0.1:8050/


/usr/lib/python3/dist-packages/ipykernel_launcher.py:76: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.

